## preprocess

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import os
import torch.nn.functional as F
import pickle
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import Counter
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm
import time
import torch
torch.cuda.is_available()  # 如果返回True，说明PyTorch可以看到GPU
torch.cuda.device_count()  # 返回GPU的数量
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import os
import json
import psutil
import re
import pandas as pd
import numpy as np
import torch
from torch_geometric import utils
from torch_geometric.loader import NeighborLoader
from torch_geometric.data import Data


# # 获取当前进程，设置进程亲和性（假设你想使用CPU 17）
p = psutil.Process(os.getpid())
p.cpu_affinity([16])

%matplotlib inline

/home/cch/.conda/envs/cch_TFE_GNN/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import hashlib
def stringtomd5(originstr):
    originstr = originstr.encode("utf-8")
    signaturemd5 = hashlib.sha256()
    signaturemd5.update(originstr)
    return signaturemd5.hexdigest()

In [3]:


# /home/cch/TraceSentinel/data/optc/benign/20-23Sep19/concat_SysClient0051.systemia.com.txt
with open("../data/optc/evaluation/23Sep19-red/malicious_SysClient0201.systemia.com.txt", 'r') as file:
    content = [json.loads(line) for line in file]




def Extract_Semantic_Info(event):
    object_type = event['object']
    properties = event['properties']

    label_mapping = {
        "PROCESS": ('parent_image_path', 'image_path'),
        "FILE": ('image_path', 'file_path'),
        "FLOW": ('image_path', 'dest_ip', 'dest_port', 'direction'),
        'MODULE': ('image_path', 'module_path')
    }

    label_keys = label_mapping.get(object_type, None)
    if label_keys:
        labels = [properties.get(key) for key in label_keys]
        # print(labels)
        if all(labels):
            if object_type=="PROCESS":
                event["actorname"], event["objectname"] = labels[0], labels[1]
                event["command_line"] = properties.get("command_line", "None") if len(properties.get("command_line", "None")) > 70 else properties.get("command_line", "None")
                # event["command_line"] = labels[2] if len(labels[2]) > 70 else labels[2]
            elif object_type=="FILE":
                event["actorname"], event["objectname"] = labels[0], labels[1]
            elif object_type=="FLOW":
                event["actorname"], event["objectname"] = labels[0], '-'.join(labels[1:3])
                event['action'] = labels[3]
            elif object_type == "MODULE":
                event["actorname"], event["objectname"] = labels[0], labels[1]
            else:
                pass
            return event
    return None


def Sentence_Construction(entry):
    action = entry["action"]
    properties = entry['properties']
    object_type = entry['object']
    phrase=[]
    try:
        if object_type=="PROCESS":
            phrase.append(properties.get("parent_image_path", "N/A"))
            phrase.append(action)
            phrase.append(properties.get("image_path", "N/A"))
            # phrase.append(properties.get("command_line", "N/A"))
        elif object_type=="FILE":
            phrase.append(properties.get("image_path", "N/A"))
            phrase.append(action)
            phrase.append(properties.get("file_path", "N/A"))
        elif object_type=="FLOW":
            phrase.append(properties.get("image_path", "N/A"))
            phrase.append(action)
            dest_ip = str(properties.get("dest_ip", "N/A"))
            dest_port = str(properties.get("dest_port", "N/A"))
            phrase.append(dest_ip+'-'+dest_port)
            # phrase.append(properties.get("src_ip", "N/A"))
            # phrase.append(properties.get("src_port", "N/A"))
            # phrase.append(properties.get("dest_ip", "N/A"))
            # phrase.append(properties.get("dest_port", "N/A"))
            # phrase.append(properties.get("direction", "N/A"))
        elif object_type=="MODULE":
            phrase.append(properties.get("image_path", "N/A"))
            phrase.append(action)
            phrase.append(properties.get("module_path", "N/A"))
        else:
            pass
    except KeyError:
        phrase = []

    return phrase

def is_valid_entry(entry):
    valid_objects = {'PROCESS', 'FILE', 'FLOW', 'MODULE'}
    invalid_actions = {'START', 'TERMINATE', 'OPEN'}

    object_valid = entry['object'] in valid_objects
    action_valid = entry['action'] not in invalid_actions
    actor_object_different = entry['actorID'] != entry['objectID']

    return object_valid and action_valid and actor_object_different

def Traversal_Rules(data):
    filtered_data = []

    for entry in data:
        if is_valid_entry(entry):
            filtered_data.append(entry)

    return filtered_data


def transform(text):
    labeled_data = [event for event in (Extract_Semantic_Info(x) for x in text) if event]
    data = Traversal_Rules(labeled_data)


    phrases = [Sentence_Construction(x) for x in data if Sentence_Construction(x)]
    for datum, phrase in zip(data, phrases):
        datum['phrase'] = phrase

    # data = sorted(data, key=lambda x: x.get('timestamp'))

    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'].str[:19], format='%Y-%m-%dT%H:%M:%S')
    df.sort_values(by='timestamp', inplace=True)
    df = df.reset_index(drop=True)

    return df


df = transform(content)
df.head()

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,phrase,command_line
0,inbound,359a1a81-783b-4828-b80b-2bae46c7dbfc,SysClient0201.systemia.com,97973fdf-5e9d-49f9-a9cc-6ea7d817d65f,FLOW,b1f76c7c-c38e-477a-8cde-66c3e26cc0c4,864,560,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '4', 'dest_ip': '224.0.0.252'...",-1,2019-09-23 09:06:13,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.252-5355,[\Device\HarddiskVolume1\Windows\system32\svch...,NaN
1,inbound,359a1a81-783b-4828-b80b-2bae46c7dbfc,SysClient0201.systemia.com,88b56ea0-185f-4c75-9aeb-abcac7313f2d,FLOW,e2201fc1-6e18-4254-b5ed-ba9cf9cbef26,864,560,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '4', 'dest_ip': '224.0.0.252'...",-1,2019-09-23 09:06:13,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.252-5355,[\Device\HarddiskVolume1\Windows\system32\svch...,NaN
2,inbound,359a1a81-783b-4828-b80b-2bae46c7dbfc,SysClient0201.systemia.com,96fd19fc-ed37-4edb-a215-fd11c5621d30,FLOW,18de7c07-d034-48f4-b7b9-e6be6ea11a49,864,560,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '4', 'dest_ip': '224.0.0.252'...",-1,2019-09-23 09:06:13,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.252-5355,[\Device\HarddiskVolume1\Windows\system32\svch...,NaN
3,inbound,359a1a81-783b-4828-b80b-2bae46c7dbfc,SysClient0201.systemia.com,8587dec6-0cd6-4df3-9c97-ba5b2b759b9c,FLOW,c75af456-14a4-48ed-8b64-239fd6bdda75,864,560,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '4', 'dest_ip': '224.0.0.252'...",-1,2019-09-23 09:06:13,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.252-5355,[\Device\HarddiskVolume1\Windows\system32\svch...,NaN
4,inbound,2f016449-765c-4896-bfca-5be85b4da8aa,SysClient0201.systemia.com,e4c1adc5-0afc-400f-85e7-5e0828191670,FLOW,70b6a611-3725-451b-aeda-3b9b10c0d6fe,2236,1472,NT AUTHORITY\SYSTEM,"{'acuity_level': '4', 'dest_ip': '225.0.0.1', ...",-1,2019-09-23 09:06:14,\Device\HarddiskVolume1\Python27\python.exe,225.0.0.1-5000,"[\Device\HarddiskVolume1\Python27\python.exe, ...",NaN


In [4]:
def proprocess_df(df):
    # 指定列转换为小写
    columns_to_lower = ['action', 'object', 'actorname', 'objectname', 'command_line', 'phrase']
    # df[columns_to_lower] = df[columns_to_lower].applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # 使用apply和lambda函数转换列中的值
    df[columns_to_lower] = df[columns_to_lower].applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # 对'phrase'列中的每个元素（列表中的字符串）应用小写转换
    df['phrase'] = df['phrase'].apply(lambda x: [item.lower() if isinstance(item, str) else item for item in x])

    node2name = {}
    # 首先遍历create事件，提取进程名
    extract_process_name = df[(df['object'] == 'process') & (df['action'] == 'create')]
    for row in extract_process_name.itertuples():
        nodeid = row.objectID
        processname = row.objectname
        if nodeid not in node2name:
            node2name[nodeid] = processname
    for row in df[(df['object'] == 'process') & (df['action'] != 'create')].itertuples():
        nodeid = row.objectID
        processname = row.objectname
        if nodeid not in node2name:
            node2name[nodeid] = processname
    # 使用 map 方法将 node2name 字典中的值应用到objectname列
    # fillna(df['objectname']) 保证如果没有对应的 nodeid，在不改变原来的值的情况下保持 objectname 原样
    df['objectname'] = df['objectID'].map(node2name).fillna(df['objectname'])
    df['actorname'] = df['actorID'].map(node2name).fillna(df['actorname'])


    new_path = {}
    for row in df[(df['object'] == 'file') & (df['action'] == 'rename')].itertuples():
        nodeid = row.objectID
        new_pathname = row.properties.get('new_path', 'None')
        if nodeid not in new_path:
            new_path[nodeid] = new_pathname

    df['objectname'] = df['objectID'].map(new_path).fillna(df['objectname'])

    # 去除重复事件
    df.drop_duplicates(subset=['action', 'actorID', 'objectID', 'object', 'actorname', 'objectname', 'pid', 'ppid'], inplace=True)
    df = df.reset_index(drop=True)

    # 根据进程名称和pid生成新的索引
    for index, row in df.iterrows():
        if row['object'] == 'process':
            # 对于 'process' 类型
            src_id = stringtomd5(row['actorname'] + str(row['ppid']))
            dest_id = stringtomd5(row['objectname'] + str(row['pid']))
        elif row['object'] in ['flow', 'file', 'module']:
            # 对于 'flow' 或 'file' 类型
            src_id = stringtomd5(row['actorname'] + str(row['pid']))
            dest_id = stringtomd5(row['objectname'])
        else:
            # 如果其他类型的 object
            src_id = None
            dest_id = None
        
        # 将结果存入新的列
        df.at[index, 'src_id'] = src_id
        df.at[index, 'dest_id'] = dest_id

    # 去除重复事件
    df.drop_duplicates(subset=['action', 'src_id', 'dest_id', 'object', 'actorname', 'objectname'], inplace=True)
    df = df[df['src_id'] != df['dest_id']]
    df = df.reset_index(drop=True)
    return df

In [5]:
df = proprocess_df(df)

In [6]:
df = df[df['object'] != "module"]
read_df = df[df['action'] == 'read']
write_actions = ['write', 'modify', 'create', 'rename']

# 筛选出写操作的行，并排除由 system 和 dwm.exe 修改的文件
# write_df = df[(df['action'].isin(write_actions)) & (~df['actorname'].isin(['system', 'dwm.exe']))]
sys_dwm = df[df['actorname'].isin(['system', 'dwm.exe'])]['dest_id']

write_df = df[df['action'].isin(write_actions)]



# write_object_ids = set(write_df['dest_id'])
write_object_ids = set(write_df['dest_id']) - set(sys_dwm)
readonly_df = read_df[~read_df['dest_id'].isin(write_object_ids)]
readonly_df_ids = readonly_df['dest_id']
df = df[~df['dest_id'].isin(readonly_df_ids)]

In [7]:
# 从df中去除名为\device\harddiskvolume1\windows\system32\svchost.exe和\device\harddiskvolume1\windows\system32\conhost.exe的节点
exclude_values = [
    '\device\harddiskvolume1\windows\system32\svchost.exe',
    '\device\harddiskvolume1\windows\system32\conhost.exe',
    'svchost.exe',
    'consent.exe'
]

# 过滤掉包含指定值的行
df = df[~df['actorname'].isin(exclude_values) & ~df['objectname'].isin(exclude_values)]
df = df.reset_index(drop=True)

In [8]:
# 找到所有 create 操作的父进程和子进程
parent_child_df = df[(df['action'] == 'create') & (df['object'] == 'process')]

# 提取所有父进程和子进程的 ID
parent_ids = set(parent_child_df['src_id'])  # 父进程 ID
child_ids = set(parent_child_df['dest_id'])   # 子进程 ID

# 所有相关的进程 ID（即有父进程或有子进程的）
related_ids = parent_ids.union(child_ids)

# 从 df 中移除既没有父进程也没有子进程的节点
df = df[df['src_id'].isin(related_ids) | df['dest_id'].isin(related_ids)]

# 重置索引
df = df.reset_index(drop=True)

In [9]:
# 过滤掉没有cmdline的进程节点：
df = df[~((df['object'] == 'process') & (df['command_line'] == 'none'))]

In [10]:
import pandas as pd
import networkx as nx

# 假设 df 是你的 DataFrame
# 1. 筛选 object 列为 'process' 的行
df_process = df[df['object'] == 'process']

# 2. 创建一个无向图
G = nx.Graph()

# 3. 添加边 (src_id, dest_id) 到图中
for _, row in df_process.iterrows():
    G.add_edge(row['src_id'], row['dest_id'])

# 4. 获取图中的所有连通子图
connected_components = list(nx.connected_components(G))

# 5. 过滤掉节点数小于 2 的子图
filtered_components = [comp for comp in connected_components if len(comp) > 2]

# 6. 获取所有保留的节点
valid_nodes = set(node for component in filtered_components for node in component)

# 7. 根据 valid_nodes 过滤原 DataFrame 中的行
# df_filtered = df_process[df_process['src_id'].isin(valid_nodes) & df_process['dest_id'].isin(valid_nodes)]

# 结果：df_filtered 就是筛选后的 DataFrame
df = df[(df['src_id'].isin(valid_nodes) | df['dest_id'].isin(valid_nodes))]

In [6]:
# df = df[df['object'] != "module"]
# read_df = df[df['action'] == 'read']
# write_actions = ['write', 'modify', 'create', 'rename']
# write_df = df[df['action'].isin(write_actions)]
# write_object_ids = set(write_df['dest_id'])
# readonly_df = read_df[~read_df['dest_id'].isin(write_object_ids)]
# readonly_df_ids = readonly_df['dest_id']
# df = df[~df['dest_id'].isin(readonly_df_ids)]

In [11]:
df.tail()

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,phrase,command_line,src_id,dest_id
6753,create,70a5d27e-5ecb-43e9-9554-13c15f1a7ed2,SysClient0201.systemia.com,75f954d0-89e5-4209-bd2c-f1c5e4b95d77,process,765dd0ae-cb99-445a-98e0-700fda47a83c,5684,3404,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': 'schtask...",-1,2019-09-23 15:41:11,\device\harddiskvolume1\windows\system32\cmd.exe,schtasks.exe,[\device\harddiskvolume1\windows\system32\cmd....,"schtasks /query /tn ""kickoffautologinrandom""",2b556ee50f47a0569028ccf2929e6f8cb2e4a661f826c8...,ddcd8a43219887880ce76802fc463cd2f285d91c049541...
6754,create,bf9af929-8abd-4fb1-ab2e-1f6e36a82b69,SysClient0201.systemia.com,18684d35-9e0b-433e-81ad-c7e53de12fb9,process,56391091-1cf5-4d1b-aee7-c28b743db3c6,5700,3536,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': 'C:\Wind...",-1,2019-09-23 15:41:11,\device\harddiskvolume1\python27\python.exe,\device\harddiskvolume1\windows\system32\cmd.exe,"[\device\harddiskvolume1\python27\python.exe, ...","c:\windows\system32\cmd.exe /c ""schtasks /dele...",fde66b35e2578280f1e19f47a008c0775d1c0643eed397...,7f4628a0df2719ec5607d37812ee8c29acf657fb0a38b5...
6755,create,9018c92c-b098-4c69-a681-208b11ac649b,SysClient0201.systemia.com,6accfb6e-7c99-4ba9-9c63-d65f8ee7963c,process,49c65442-58bf-45a0-841d-b7f9140b451d,980,3768,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': 'schtask...",-1,2019-09-23 15:41:11,\device\harddiskvolume1\windows\system32\cmd.exe,schtasks.exe,[\device\harddiskvolume1\windows\system32\cmd....,"schtasks /query /tn ""kickoffautologinrandom""",c4692c6b552575dd4810ec3a2e263c120dc9d0f9a6561a...,581539295a9003f82559c33bd3db5d53a9b7696d3e25fe...
6756,create,329a0883-f669-44f3-bf2a-2b420da1650c,SysClient0201.systemia.com,f227ec73-e547-4bed-b1db-7edff1eddbdf,process,ffeefd5a-d18f-48f1-82e7-db1673209d2d,3308,560,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': 'C:\Wind...",-1,2019-09-23 15:41:53,\device\harddiskvolume1\windows\system32\servi...,\device\harddiskvolume1\windows\servicing\trus...,[\device\harddiskvolume1\windows\system32\serv...,c:\windows\servicing\trustedinstaller.exe,6ab226c489ee0047b0ee5f7991fdd476dbcc3741c4d41a...,30d4a1095d7bec7c77313436880a32ff53d292c372a45c...
6757,modify,d46f3eea-d5b1-495f-9a7d-2bcdfd24764e,SysClient0201.systemia.com,35c91479-3e25-4701-9d1e-43311f2bc602,file,b378e939-ab8b-4f4b-8fde-1f8f628c89bb,4684,1932,SYSTEMIACOM\zleazer,"{'acuity_level': '3', 'file_path': '\Device\Ha...",5316,2019-09-23 15:42:57,\device\harddiskvolume1\program files (x86)\co...,\device\harddiskvolume1\programdata\adobe\read...,[\device\harddiskvolume1\program files (x86)\c...,NaN,0709cb7a5371f63e8225489a03ff1de499f59d7bf760c7...,a6a2cf84403cd91229256e9bd1ada66ed1cae4f33973b8...


## word2vec

In [12]:
import json
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

class EpochSaver(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('../trained_weights/optc/cmdline_attack1_w2v.model')
        self.epoch += 1

class EpochLogger(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

def prepare_sentences(df):
    nodes = {}
    for index, row in df.iterrows():
        for key in ['src_id', 'dest_id']:
            node_id = row[key]
            nodes.setdefault(node_id, []).extend(row['phrase'])
    return list(nodes.values())

def train_word2vec_model(df):
    phrases = prepare_sentences(df)

    logger = EpochLogger()
    saver = EpochSaver()
    word2vec = Word2Vec(sentences=phrases, vector_size=64, window=5, min_count=2, workers=8, epochs=50, callbacks=[saver, logger])

    return word2vec

In [6]:
# train_word2vec_model(df)

In [13]:
# 训练过程见optc_parser3
word2vec_weights = '../trained_weights/optc/cmdline_attack1_w2v.model'
w2vmodel = Word2Vec.load(word2vec_weights)

## featurize

In [14]:
import numpy as np


def infer(word):
    if word in  w2vmodel.wv:
        word_embeddings = w2vmodel.wv[word]
    else:
        word_embeddings = np.zeros(64)

    output_embedding = torch.tensor(word_embeddings, dtype=torch.float)
    output_embedding = output_embedding.detach().cpu().numpy()
    return output_embedding


def Featurize(df):
    # dummies = {'process': 0, 'flow': 1, 'file': 2, 'module': 3}
    dummies = {'process': 0, 'flow': 1, 'file': 2}

    nodes = {}
    labels = {}
    lblmap = {}
    neimap = {}
    edges = []

    for index, row in df.iterrows():
        actor_id, object_id = row['src_id'], row["dest_id"]
        object_type = row['object']

        if actor_id not in nodes:
            nodes[actor_id] = row['actorname']
        if object_id not in nodes:
            nodes[object_id] = row['objectname']

        # nodes.setdefault(actor_id, []).extend(row['phrase'])
        # nodes.setdefault(object_id, []).extend(row['phrase'])

        labels[actor_id] = dummies.get('process', -1)
        labels[object_id] = dummies.get(object_type, -1)

        if actor_id not in lblmap:
            lblmap[actor_id] = row['actorname']
        if object_id not in lblmap:
            lblmap[object_id] = row['objectname']

        neimap.setdefault(actor_id, set()).add(row['objectname'])
        neimap.setdefault(object_id, set()).add(row['actorname'])

        edge_type = row['action']
        edges.append((actor_id, object_id, edge_type))

    features, feat_labels, edge_index = [], [], [[], []]
    node_index = {}

    for node, nodename in nodes.items():
        features.append(infer(nodename))
        feat_labels.append(labels[node])
        node_index[node] = len(features) - 1

    for src, dst, action in edges:
        if action=='read' or action=='inbound':
            edge_index[0].append(node_index[dst])
            edge_index[1].append(node_index[src])
        elif action=='create':
            edge_index[0].append(node_index[src])
            edge_index[1].append(node_index[dst])
            edge_index[0].append(node_index[dst])
            edge_index[1].append(node_index[src])
        else:
            edge_index[0].append(node_index[src])
            edge_index[1].append(node_index[dst])

    mapp = list(node_index.keys())

    return features, np.array(feat_labels), edge_index, mapp, lblmap, neimap

nodes,labels,edges,mapp,lbl,nemap = Featurize(df)

In [15]:
len(nodes),len(labels),len(mapp)

(4923, 4923, 4923)

In [16]:
# 定义一个字典，存储每个节点对应的特征向量
nodes_vec = {}
for i, j in zip(mapp, nodes):
    nodes_vec[i] = j

In [61]:
# 交换源节点和目标节点，因为信息汇聚是由源节点汇聚到目标节点
# edges[0], edges[1] = edges[1], edges[0]

In [16]:
# infer('dwm.exe')

In [13]:
# 从df中去除名为\device\harddiskvolume1\windows\system32\svchost.exe和\device\harddiskvolume1\windows\system32\conhost.exe的节点
# exclude_values = [
#     '\device\harddiskvolume1\windows\system32\svchost.exe',
#     '\device\harddiskvolume1\windows\system32\conhost.exe',
#     'svchost.exe',
#     'consent.exe'
# ]

# # 过滤掉包含指定值的行
# df = df[~df['actorname'].isin(exclude_values) & ~df['objectname'].isin(exclude_values)]
# df = df.reset_index(drop=True)

## lowest_ancestor_find

In [17]:
df_process_lineage = df[(df['action'] == 'create') & (df['object'] == 'process')]

# 记录每个进程的父进程及其时间戳和 cmdline 信息
process_parents = {}

# 进程谱系树生成
import networkx as nx

g = nx.MultiDiGraph()
node_map = {}
# count_node = 0
for index, row in df_process_lineage.iterrows():
    src, source_name, dst, dstname, etype, object_type = row['src_id'], row["actorname"],  row['dest_id'], row['objectname'], row['action'], row['object']
    command_line = row['command_line']
    if src not in node_map:
        node_map[src] = src
        g.add_node(src, type="process", pname=source_name)
    
    if dst not in node_map:
        node_map[dst] = dst
        if source_name==dstname:
            g.add_node(dst, type=object_type, pname=dstname)
        else:
            g.add_node(dst, type=object_type, pname=dstname)
    
    if g.has_edge(node_map[dst], node_map[src]):
        pass
    else:
        if g.has_edge(node_map[src], node_map[dst]):
            pass
        elif etype=='create':
            g.add_edge(node_map[src], node_map[dst], type=etype, commandline=command_line)
        else:
            pass
    if dst not in process_parents:
        process_parents[dst] = []
    process_parents[dst].append((src, command_line))

def find_lowest_common_ancestor(graph, target_pid):
    """
    查找某个进程的具有多个子进程的最低祖先。
    :param graph: NetworkX图对象
    :param target_pid: 目标进程ID
    :return: 具有多个子进程的最低祖先进程ID
    """
    # 获取目标进程的父进程
    def get_parent(pid):
        parents = process_parents.get(pid, [])
        if parents:
            return parents[0][0]  # 假设每个进程有一个父进程
        return None
    
    # 遍历并查找具有多个子进程的父进程
    def has_multiple_children(pid):
        children = list(graph.successors(pid))
        return len(children) > 1  # 如果父进程有多个子进程
    
    # 从目标进程开始，逐步向上遍历父进程
    current_pid = target_pid
    
    while True:
        parent_pid = get_parent(current_pid)
        if parent_pid is None:
            return None  # 找不到父进程，说明已经是根节点
        if has_multiple_children(parent_pid):
            # return parent_pid
            return current_pid  # 找到具有多个子进程的最低祖先
        current_pid = parent_pid  # 向上查找父进程

# 存储每个进程的最低祖先
process_lca_dict = {}



# 遍历所有进程，查找其最低祖先并存储
for node in g.nodes:
    lca = find_lowest_common_ancestor(g, node)
    if lca is not None:
        process_lca_dict[node] = process_parents.get(lca, 'meiyou')[0][1]

In [27]:
# len(process_lca_dict)

## model

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class GNNWithRootEmbedding(nn.Module):
    def __init__(self, num_features, process_lca_dict, node_id_to_hash, node_hash_to_vec):
        super(GNNWithRootEmbedding, self).__init__()
        
        self.node_hash_to_vec = node_hash_to_vec # 存储节点的uuid到其初始向量的映射，其中uuid是hash值
        self.node_id_to_hash = node_id_to_hash # 存储节点的batch.n_id到节点uuid的映射，其中batch.n_id的数值，例如1，2，3，其中uuid是hash值
        self.process_lca_dict = process_lca_dict  # 存储节点与其根节点ID的映射
        self.num_features = num_features  # 节点特征维度
        
        # 第一层图卷积，输入维度为特征 + 根节点特征维度
        self.conv1 = SAGEConv(num_features, 128, normalize=True)  # 用于邻居节点特征的聚合
        # 第二层图卷积
        self.conv2 = SAGEConv(128, 64, normalize=True)
        
        # 用于根节点特征的变换矩阵W2
        self.root_transform = nn.Linear(num_features, 128)
        # 用于根节点特征的变换矩阵W3
        self.root_transform2 = nn.Linear(128, 64)

        self.linear = nn.Linear(64, 3)
    
    def root_embedding(self, node_ids):
        """
        通过根节点映射字典，直接获取根节点的特征。
        :param node_ids: 当前批次中的节点ID
        :return: 返回每个节点对应的根节点嵌入特征
        """
        # 获取节点对应的hash值
        node_hash = [self.node_id_to_hash[node_id] for node_id in node_ids]
        # 获取节点对应的根节点特征（通过process_lca_dict获得根节点ID）
        root_ids = [self.process_lca_dict.get(node_id, "meiyougengjiedian") for node_id in node_hash]
        
        # 在这里你可以调用任意方法返回根节点的特征
        # 假设根节点特征向量已经通过某种方式计算并存储，返回对应的特征
        # get_origin_embedding 函数假设返回根节点的嵌入特征
        # return torch.stack([torch.tensor(self.node_hash_to_vec.get(root_id, torch.zeros(self.num_features))) for root_id in root_ids], dim=0)
        return torch.stack([torch.tensor(infer(root_id), dtype=torch.float) for root_id in root_ids], dim=0)

    def forward(self, x, edge_index, batch_node_ids):

        x = self.encode(x, edge_index, batch_node_ids)

        x = self.linear(x)

        return F.softmax(x, dim=1)
        
    def encode(self, x, edge_index, batch_node_ids):
        """
        :param x: 节点特征矩阵
        :param edge_index: 边索引
        :param batch_node_ids: 当前批次的节点ID
        """
        # 获取当前批次节点的根节点特征
        root_embeds = self.root_embedding(batch_node_ids)

        # 第一步，聚合邻居特征
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        # 第二步，计算根节点特征的影响，根节点特征通过线性变换 W2
        root_embeds_transformed = self.root_transform(root_embeds.to(device))
        root_embeds_transformed = F.relu(root_embeds_transformed)
        root_embeds_transformed = F.dropout(root_embeds_transformed, p=0.2, training=self.training)

        # 最后，邻居特征加上根节点变换后的特征
        x = x + root_embeds_transformed

        # 第二层图卷积
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)


        root_embeds_transformed2 = self.root_transform2(root_embeds_transformed)
        root_embeds_transformed2 = F.relu(root_embeds_transformed2)
        root_embeds_transformed2 = F.dropout(root_embeds_transformed2, p=0.2, training=self.training)

        x = x + root_embeds_transformed2

        return x

In [19]:
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GNNWithRootEmbedding(64, process_lca_dict, mapp, nodes_vec).to(device)
# if not gnnTrain:
#     model.load_state_dict(torch.load(gnn_weights))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [20]:
from sklearn.utils import class_weight

l = np.array(labels)
class_weights = class_weight.compute_class_weight(class_weight = "balanced",classes = np.unique(l),y = l)
class_weights = torch.tensor(class_weights,dtype=torch.float).to(device)
criterion = CrossEntropyLoss(weight=class_weights,reduction='mean')

graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))

In [21]:
graph.num_nodes

4923

In [22]:
# 定义训练节点为process节点
train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool) 
for i , j in enumerate(labels):
    if j == 0:
        train_mask[i] = True
graph.train_mask = train_mask
print(sum(train_mask))

tensor(1458)


In [23]:
loader = NeighborLoader(graph, num_neighbors=[-1, -1], batch_size=5000, shuffle=True, input_nodes=graph.train_mask)

In [19]:
# for i in range(len(edges[0])):
#     if edges[1][i]==11752:
#         print(edges[0][i], edges[1][i])

In [28]:
# num_baatch_nid = 0
# for i,batch in enumerate(loader):
#     print(batch.n_id)
#     num_baatch_nid=num_baatch_nid+len(batch.n_id)
# print(num_baatch_nid)

In [21]:
# mapp[14457]

In [24]:
def evaluate_model(batch):
    model.eval()
    with torch.no_grad():
        predictions = model(batch.x, batch.edge_index, batch.n_id)[:batch.batch_size]
        pred_labels = predictions.argmax(dim=1)
        y = batch.y[:batch.batch_size]
        correct_predictions = int((pred_labels == y).sum())
    return correct_predictions

In [25]:
gnn_weights = "../trained_weights/optc/cmdline_attack1.pth"

In [26]:
model.load_state_dict(torch.load(gnn_weights))

<All keys matched successfully>

In [27]:
total_loss = total_correct = total_nodes = 0

for batch in loader:
    num_nodes = batch.batch_size
    total_nodes += num_nodes
    total_correct += evaluate_model(batch)
# average_loss = total_loss / total_nodes
accuracy = total_correct / total_nodes

print(f"Accuracy: {accuracy:.5f}")

Accuracy: 0.78121


In [28]:
model.eval()
for batch in loader:
    with torch.no_grad():
        predictions = model(batch.x, batch.edge_index, batch.n_id)[:batch.batch_size]
        pred_labels = predictions.argmax(dim=1)
        y = batch.y[:batch.batch_size]
        # print(pred_labels.shape)
        # print(batch.y.shape)
        incorrect_nodes  = pred_labels != y

        process_entity_ids = batch.n_id[:batch.batch_size]
        incorrect_node_ids = batch.n_id[:batch.batch_size][incorrect_nodes]

In [29]:
len(incorrect_node_ids)

319

In [30]:
identified_ids = {mapp[idx] for idx in incorrect_node_ids}

In [31]:
EntitySet = {mapp[idx] for idx in process_entity_ids}

In [32]:
len(EntitySet)

1458

In [33]:
infer("\device\harddiskvolume1\program files\windows defender\msascuil.exe")

array([-0.0601708 ,  0.11621949,  0.65388966,  1.0378107 , -0.2651753 ,
        0.9758369 , -0.6021893 , -1.281443  , -0.20467827,  1.054397  ,
       -0.815483  ,  1.5322576 , -0.17407279, -0.46693915,  0.5225777 ,
        0.13027744, -0.40977493,  0.10183879,  0.26123926, -0.82700014,
       -1.8016226 , -0.6519543 , -0.8772601 ,  0.7415297 , -0.7743878 ,
        0.09844316,  1.1949521 , -1.0157355 ,  0.1010242 , -0.18114488,
        0.88763005, -1.4479128 ,  0.27544802,  0.15120733,  1.563829  ,
        0.17456675, -0.18218939,  0.07466054, -0.4780742 , -0.33879074,
       -1.0261356 , -0.31914505, -1.2747583 ,  1.2778543 ,  2.9707186 ,
       -0.3275396 ,  0.66307193, -0.04112538, -1.0856901 , -0.00546277,
        0.53423345,  2.2661161 ,  0.9132522 , -0.4081871 ,  0.39664242,
        0.61517483, -0.03994675,  1.8733113 ,  1.5452766 ,  0.7741029 ,
       -0.6759333 , -0.8168321 ,  0.6854603 ,  0.17262891], dtype=float32)

In [34]:
for i in incorrect_node_ids:
    if lbl[mapp[i]] !="\device\harddiskvolume1\windows\system32\conhost.exe" and lbl[mapp[i]] !="\device\harddiskvolume1\windows\system32\svchost.exe":
        print(lbl[mapp[i]])

\device\harddiskvolume1\windows\system32\searchprotocolhost.exe
\device\harddiskvolume1\windows\system32\ping.exe
\\?\c:\program files (x86)\mozilla firefox\firefox.exe
\device\harddiskvolume1\windows\system32\searchprotocolhost.exe
\device\harddiskvolume1\windows\system32\ping.exe
\device\harddiskvolume1\program files (x86)\common files\adobe\arm\1.0\adobearm.exe
\device\harddiskvolume1\windows\system32\ping.exe
\device\harddiskvolume1\windows\system32\ping.exe
taskkill.exe
\device\harddiskvolume1\windows\system32\ping.exe
\device\harddiskvolume1\windows\regedit.exe
\device\harddiskvolume1\windows\system32\searchprotocolhost.exe
\device\harddiskvolume1\windows\system32\ping.exe
\device\harddiskvolume1\windows\system32\ping.exe
\device\harddiskvolume1\windows\system32\ping.exe
\device\harddiskvolume1\windows\system32\ping.exe
\device\harddiskvolume1\windows\system32\ping.exe
\device\harddiskvolume1\windows\system32\ping.exe
\device\harddiskvolume1\windows\system32\cmd.exe
\device\hardd

In [35]:
for i in incorrect_node_ids:
    if lbl[mapp[i]] !="\device\harddiskvolume1\windows\system32\conhost.exe" and lbl[mapp[i]] !="\device\harddiskvolume1\windows\system32\svchost.exe":
        print(mapp[i])

...


In [34]:
# df['object'].value_counts()

## evaluation

In [36]:
def load_ground_truth(gt_file):
    with open(gt_file, 'r') as file:
        gt_nodes = set(file.read().split())
    return gt_nodes

In [37]:
gt_nodes = load_ground_truth('../data/optc/attack1_groundtruth.txt')

In [ ]:
# 假设 gt_nodes 是一个包含 event_src 和 event_dest 值的集合
# gt_nodes = [...]  # 示例集合

# 用来存储对应的 actorid 和 objectid
actorid_objectid_pairs = []

# 遍历 gt_nodes 集合
for n in gt_nodes:
    # 筛选 df 中 event_src 或 event_dest 与 node 匹配的行
    matching_rows = df[(df['src_id'] == n)]
    matching_rows2 = df[(df['dest_id'] == n)]
    # 如果找到匹配的行，获取 actorid 和 objectid
    if not matching_rows.empty:
        actorid = matching_rows['actorID'].values[0]  # 只取第一个匹配的 actorid
        # objectid = matching_rows['objectid'].values[0]  # 只取第一个匹配的 objectid
        actorid_objectid_pairs.append(actorid)
        continue
    if not matching_rows.empty:
        objectid = matching_rows['objectID'].values[0]  # 只取第一个匹配的 actorid
        # objectid = matching_rows['objectid'].values[0]  # 只取第一个匹配的 objectid
        actorid_objectid_pairs.append(objectid)
    if matching_rows.empty:
        print('aaa')

# 打开文件并写入
with open('/home/cch/TraceSentinel/data/optc/attack1_groundtruth_2025.txt', 'w') as file:
    for item in actorid_objectid_pairs:
        file.write(item + '\n')  # 每个元素写入新的一行    

In [35]:
# actorid_objectid_pairs

In [36]:
# df[df['objectID'] == 'af6b49d5-f648-41a4-946d-d92b174bae47']['objectname'].iloc[0]

In [39]:
melicious_name = []
for i in gt_nodes:
    melicious_name.append(lbl[i])

In [40]:
len(identified_ids), len(gt_nodes), len(EntitySet)

(319, 218, 1458)

In [ ]:
def calculate_metrics(TP, FP, FN, TN):
    FPR = FP / (FP + TN) if FP + TN > 0 else 0
    TPR = TP / (TP + FN) if TP + FN > 0 else 0

    prec = TP / (TP + FP) if TP + FP > 0 else 0
    rec = TP / (TP + FN) if TP + FN > 0 else 0
    fscore = (2 * prec * rec) / (prec + rec) if prec + rec > 0 else 0

    return prec, rec, fscore, FPR, TPR

def Get_Adjacent(ids, mapp, edges, hops):
    if hops == 0:
        return set()
    
    neighbors = set()
    for edge in zip(edges[0], edges[1]):
        if any(mapp[node] in ids for node in edge):
            neighbors.update(mapp[node] for node in edge)

    if hops > 1:
        neighbors = neighbors.union(Get_Adjacent(neighbors, mapp, edges, hops - 1))
    
    return neighbors

def helper(MP, all_pids, GP):
    """
    输入:
    MP预测错误的节点id的集合
    all_pids所有节点id的集合
    GP是grundtruth的节点id的集合
    edges是边的合集
    mapp是节点id的合集
    """
    TP = MP.intersection(GP)
    FP = MP - GP
    FN = GP - MP
    TN = all_pids - (GP | MP)


    # two_hop_gp找到GP的1跳邻居节点
    two_hop_gp = Get_Adjacent(GP, mapp, edges, 1)
    # two_hop_tp找到TP的1跳邻居节点
    two_hop_tp = Get_Adjacent(TP, mapp, edges, 1)

    # two_hop_fp找到FP的1跳邻居节点
    two_hop_fp = Get_Adjacent(FP, mapp, edges, 1)
    for i in list(FP):
        a = set()
        a.add(i)
        one_hop_fp = Get_Adjacent(a, mapp, edges, 1)
        # print(a)
        for j in one_hop_fp:
            if lbl[j] in melicious_name:
                FP.discard(i)
    

    FPL = FP - two_hop_gp
    TPL = TP.union(FN.intersection(two_hop_tp))
    FN = FN - two_hop_tp

    TPl, FPl, FNl, TNl = len(TPL), len(FPL), len(FN), len(TN)



    # TPl, FPl, FNl, TNl = len(TP), len(FP), len(FN), len(TN)
    print(TPl, FPl, FNl, TNl)
    prec, rec, fscore, FPR, TPR = calculate_metrics(TPl, FPl, FNl, TNl)
    print(f"Precision: {round(prec, 2)}, Recall: {round(rec, 2)}, Fscore: {round(fscore, 2)}")
    return FP, FN

In [1]:
# FP, FN = helper(identified_ids, EntitySet , gt_nodes)

In [46]:
FP, FN = helper(identified_ids, EntitySet , gt_nodes)

218 37 0 1138
Precision: 0.85, Recall: 1.0, Fscore: 0.92


In [50]:
for i in FN:
    print(lbl[i])

In [51]:
for i in FN:
    print(i)

In [ ]:
for i in FP:
    print(lbl[i])

In [ ]:
aaa = r'\device\harddiskvolume1\windows\system32\services.exe'
aaa

In [ ]:
infer(r'ping -w 2000 -n 2 127.0.0.1')

In [ ]:
for i in FP:
    print(i)